In [21]:
import pandas as pd
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import os
import numpy as np
import random 
from collections import Counter
from spacy.tokens import DocBin

In [22]:
nlp = spacy.load("en_core_web_lg")

In [23]:
# set working directory to location for gold standard labels dataset
os.chdir('C:/Users/asanchez/OneDrive - Gongos, Inc/Desktop')

In [24]:
# manually labeled the excel file for relevant vs non-relevant 
data_file = '02_Relevant_Sessions_Detail_part1_cleaning.xlsx'
df = pd.read_excel(data_file,
                   usecols=['keyword', 'Keyword_Relevant'])

In [25]:
df = df.dropna()
df = df.drop_duplicates(subset=['keyword'])
#df.to_csv("train_file.csv")


In [26]:
# significant class imbalance
print(Counter(df.Keyword_Relevant))

Counter({0.0: 405, 1.0: 76})


In [27]:
# consider using SMOTEENN

In [28]:
df['tuples'] = df[['keyword', 'Keyword_Relevant']].apply(tuple, axis=1)

In [29]:
df['tuples'].iloc[random.randint(0, len(df))]

('country club driving range titleist fitting', 0.0)

In [30]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples = True):
        if label == 1:
            doc.cats['relevant'] = 1
            doc.cats['not_relevant'] = 0

        else:
            doc.cats['relevant'] = 0
            doc.cats['not_relevant'] = 1
        docs.append(doc)
    return docs

In [31]:
from sklearn.model_selection import train_test_split
X = df['tuples']
train, test = train_test_split(X, shuffle="y", test_size=0.25)
train.shape, test.shape

((360,), (121,))

In [32]:
train_docs = make_docs(train)
#binerize data 
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

valid_docs = make_docs(test)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")

## Train spacy model

In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./output

## spacy model training test

In [38]:
os.chdir('O:/Amazon/2021/AMZ210470 Major Appliances/1_Behavioral Data Analysis/output')
data_file = '02_Relevant Sessions_Detail.csv'
df = pd.read_csv(data_file, encoding="ISO-8859-1")

In [40]:
titles = [_ for _ in df['keyword_search'].dropna().str.lower()]
text = random.choice(titles)
text

'front load washer and dryer'

In [41]:
# this is the location of the model
os.chdir("C:/Users/asanchez/OneDrive - Gongos, Inc/Desktop/")
# load the model
nlp = spacy.load("output/model-best")

In [42]:
doc = nlp(text)

In [43]:
# test the model on random text from dataset 
print(text, doc.cats)

front load washer and dryer {'relevant': 0.7234464287757874, 'not_relevant': 0.2780964970588684}
